In [3]:
# Importation des bibliothèques nécessaires
import os
import pickle
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
# Étape 1 : Chargement des données (simplifié ici)
def load_isot_dataset(fake_file, true_file):
    """Charge les fichiers de données et retourne un DataFrame pour les données de fausses et vraies nouvelles."""
    fake_data = pd.read_csv(fake_file)
    true_data = pd.read_csv(true_file)
    
    fake_data['label'] = 0  # Label 0 pour les fausses nouvelles
    true_data['label'] = 1  # Label 1 pour les vraies nouvelles
    
    return pd.concat([fake_data, true_data], ignore_index=True)

# Exemple de chargement de données (remplacez par vos vrais fichiers)
train_data = load_isot_dataset("data/Fake.csv", "data/True.csv")

# Diviser les données en caractéristiques (X) et labels (y)
X = train_data["text"]  # Le texte des articles
y = train_data["label"]  # Les labels (0 ou 1)

In [5]:
# Étape 2 : Prétraitement et extraction des caractéristiques
def extract_tfidf_features(X_train, X_test):
    """Extrait les caractéristiques TF-IDF des jeux de données."""
    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    return X_train_tfidf, X_test_tfidf

# Séparer les données en entraînement et test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Extraire les caractéristiques TF-IDF
X_train_tfidf, X_test_tfidf = extract_tfidf_features(X_train, X_test)

In [10]:
# Étape 3 : Entraîner et sauvegarder un modèle
def train_model(X_train, y_train, model_type="svm"):
    """Entraîne un modèle classique et sauvegarde le modèle."""
    if model_type == "svm":
        model = SVC(kernel="linear")
    elif model_type == "random_forest":
        model = RandomForestClassifier(n_estimators=100)
    elif model_type == "logistic_regression":
        model = LogisticRegression()

    model.fit(X_train, y_train)

    # Création du dossier 'models' s'il n'existe pas
    os.makedirs("models", exist_ok=True)

    # Sauvegarde du modèle
    with open(f"models/{model_type}.pkl", "wb") as f:
        pickle.dump(model, f)

    return model

# Entraîner un modèle SVM (par exemple)
model = train_model(X_train_tfidf, y_train, model_type="logistic_regression")

In [11]:
# Étape 4 : Charger et évaluer le modèle
def evaluate_model(X_test, y_test, model_path):
    """Évalue un modèle chargé depuis un fichier."""
    try:
        with open(model_path, "rb") as f:
            model = pickle.load(f)
    except FileNotFoundError:
        print(f"Erreur : fichier {model_path} introuvable.")
        return

    y_pred = model.predict(X_test)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Évaluer le modèle sur les données de test
evaluate_model(X_test_tfidf, y_test, "models/logistic_regression.pkl")

Accuracy: 0.9858574610244989
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      4733
           1       0.98      0.99      0.99      4247

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

Confusion Matrix:
 [[4657   76]
 [  51 4196]]
